# Volatility Forecast Dataset

In [15]:
#!pip install yfinance
# !pip install pandas_ta
# !pip install arch

In [16]:
# If yfinance throws an error then uncomment the line below to install it. (You need to install it only once so comment it again after)
import yfinance as yf
import pandas_ta as ta
import pandas as pd
import numpy as np
from arch import arch_model

In [17]:
fb, aapl, amzn, nflx, googl   = yf.Ticker("FB"), yf.Ticker("AAPL"), yf.Ticker("AMZN"), yf.Ticker("NFLX"), yf.Ticker("GOOGL")

tickers = ("fb", "aapl", "amzn","nflx", "googl")
dfs = {"fb": fb.history("20Y"), "aapl" : aapl.history("20Y"), "amzn" : amzn.history("20Y"), "nflx": nflx.history("20Y"), "googl":googl.history("20Y")}

In [18]:
for i in tickers:
    df = dfs[i]
    dfs[i].drop(columns = ["Dividends","Stock Splits"],inplace = True)
    
    MACD = df.ta.macd(close='close', fast=12, slow=26, signal=9, append=True).drop(columns=["MACDh_12_26_9","MACDs_12_26_9"])
    RSI = df.ta.rsi()
    EMA = df.ta.ema()
    MFI = df.ta.mfi()
    
    dfs[i]['Return'] = df['Close'].pct_change()
    dfs[i]['Vol-5d'] = df['Return'].rolling(window='5d').std()
    dfs[i]['MACD'] = MACD
    dfs[i]['RSI'] = RSI
    dfs[i]['EMA'] = EMA
    dfs[i]['MFI'] = MFI
    
    dfs[i].dropna(inplace = True)
    dfs[i].drop(columns = ["MACD_12_26_9","MACDh_12_26_9", "MACDs_12_26_9"], inplace = True)
    
    dfs[i]['signal'] = np.zeros(len(df))
    
    for j,time in enumerate(df.index):
    
        before = df["Return"][j-5:j].std()
        after = df["Return"][j:j+5].std()

        if after >= before:
            dfs[i]['signal'][j] = 1
    

<ipython-input-18-a6b1427f34f6>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs[i]['signal'][j] = 1
<ipython-input-18-a6b1427f34f6>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs[i]['signal'][j] = 1
<ipython-input-18-a6b1427f34f6>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs[i]['signal'][j] = 1
<ipython-input-18-a6b1427f34f6>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [20]:
for i in tickers:
    dfs[i].to_csv(i + ".csv")